# Classifying a Single Target

This notebook demonstrates the photometric classification of a single supernova (SN) from the Carnegie Supernova Project (CSP). It is a simplified example intended to outline the general classification technique introduced by González-Gaitán et al. 2014. 

Two approaches are demonstrated in this notebook. In the first, data is split into rest frame red and rest frame blue bandpasses. Fits are then performed to both the blue and red data using models for a normal and 91bg-like supernova. The resulting classification of the target is determined using the relative quality of fit (normal chi-squared - 91bg-like chi-squared) in both bandpass collections. In the second approach, fits are performed in each bandpass independently and results are seperated afterword into red and blue bands. 

The first section of this notebook is intended to introduce the general concept. The second notebook introduces some important extra-details.

#### Table of Contents:
1. <a href='#fitting_collectively'>Fitting Bands Collectively</a>: Classification performed by fitting blue and red data as seperate band sets.
1. <a href='#fitting_independently'>Fitting Bands Independently</a>: Classification performed by fitting bands independently and then seperating results into the blue and red afterword.


In [ ]:
import sys

import sncosmo
from matplotlib import pyplot as plt
from sndata.csp import dr3

sys.path.insert(0, '../')
from phot_class import models
from phot_class import utils
from phot_class.fit_funcs import simple_fit
from phot_class import classification

dr3.download_module_data()
dr3.register_filters(force=True)
models.register_sources(force=True)


## Fitting Bands Collectively <a id='fitting_collectively'></a>

We start by reading in data for an arbitrarily chosen SN. For demonstration purposes, we pick a known 91bg-like SN and drop band-passes that are in the NIR. 

In [ ]:
demo_id = '2005ke'
all_data = dr3.get_data_for_id(demo_id)
all_data = all_data[all_data['band'] != 'csp_dr3_Y']
all_data = all_data[all_data['band'] != 'csp_dr3_Ydw']
all_data = all_data[all_data['band'] != 'csp_dr3_J']
all_data = all_data[all_data['band'] != 'csp_dr3_H']


Using the spectroscopic redshift of the target, we split the photometric data into rest frame blue and red wavelengths. In practice, we may not have the redshift value and will need to split data using a photometrically determined redshift.

In [ ]:
redshift = all_data.meta['redshift']
blue_data, red_data = utils.split_data(
    all_data, dr3.band_names, dr3.lambda_effective, redshift)

print('Blue data bands:', set(blue_data['band']))
print('Red data bands :', set(red_data['band']))


Next, we fit the red and blue data using models for Hsiao and sn91bg-like SNe. Naively this results in four total light-curve fits; however, we also perform an initial fit using the complete light-curve and the Hsiao model. This additional fit helps us determine an initial guess for the time of maximum and ensures more predictable fitting results.


In [ ]:
sn91bg = sncosmo.Model(sncosmo.get_source('sn91bg', version='hsiao_phase'))
hsiao = sncosmo.Model('hsiao')

bg_params = ['t0', 'amplitude', 'x1', 'c']
hs_params = ['t0', 'amplitude']

initial_result, _ = simple_fit(all_data, hsiao, hs_params)
t0 = initial_result.parameters[1]

hsiao.set(t0=t0, z=redshift)
hsiao_bounds = {'t0': (t0 - 3, t0 + 3)}

sn91bg.set(t0=t0, z=redshift)
sn91bg_bounds = {
    't0': (t0 - 3, t0 + 3), 
    'x1': (0.65, 1.25),
    'c': (0, 1)}

blue_hsiao_result, blue_hsiao_fit = simple_fit(blue_data, hsiao, hs_params, bounds=hsiao_bounds)
red_hsiao_result, red_hsiao_fit = simple_fit(red_data, hsiao, hs_params, bounds=hsiao_bounds)

blue_sn91bg_result, blue_sn91bg_fit = simple_fit(blue_data, sn91bg, bg_params, bounds=sn91bg_bounds)
red_sn91bg_result, red_sn91bg_fit = simple_fit(red_data, sn91bg, bg_params, bounds=sn91bg_bounds)


We pause here to visually inspect the fit results.

In [ ]:
sncosmo.plot_lc(blue_data, blue_hsiao_fit)
sncosmo.plot_lc(red_data, red_hsiao_fit)
sncosmo.plot_lc(blue_data, blue_sn91bg_fit)
sncosmo.plot_lc(red_data, red_sn91bg_fit)
plt.show()


Finally, we determine the chi-squared of each fit and use them to determine a classification coordinate. If we were to repeat this process for multiple SNe and plot the resulting coordinates, we would expect 91bg like SNe to fall in the upper right quadrant and hsiao-like SNe to fall in the lower-left corner (see González-Gaitán et al. 2014 for more details). Since our example light curve is from a 91bg-like SN, we expect a positive x and y coordinate.

In [ ]:
blue_hsiao_chisq, blue_hsiao_dof = utils.calc_model_chisq(blue_data, blue_hsiao_result, blue_hsiao_fit)
red_hsiao_chisq, red_hsiao_dof = utils.calc_model_chisq(red_data, red_hsiao_result, red_hsiao_fit)

blue_sn91bg_chisq, blue_sn91bg_dof = utils.calc_model_chisq(blue_data, blue_sn91bg_result, blue_sn91bg_fit)
red_sn91bg_chisq, red_sn91bg_dof = utils.calc_model_chisq(red_data, red_sn91bg_result, red_sn91bg_fit)

print('Reduced chi-squared values:')
print(f'hsiao Blue : {blue_hsiao_chisq} / {blue_hsiao_dof} = {blue_hsiao_chisq / blue_hsiao_dof}')
print(f'hsiao Red  : {red_hsiao_chisq} / {red_hsiao_dof} = {red_hsiao_chisq / red_hsiao_dof}')
print(f'SN91bg Blue: {blue_sn91bg_chisq} / {blue_sn91bg_dof} = {blue_sn91bg_chisq / blue_sn91bg_dof}')
print(f'SN91bg Red : {red_sn91bg_chisq} / {red_sn91bg_dof} = {red_sn91bg_chisq / red_sn91bg_dof}')

coordinates = (
    (blue_hsiao_chisq / blue_hsiao_dof) - (blue_sn91bg_chisq / blue_sn91bg_dof), 
    (red_hsiao_chisq / red_hsiao_dof) - (red_sn91bg_chisq / red_sn91bg_dof)
)

print('Coordinates:', coordinates)


## Fitting Bands Independently <a id='fitting_independently'></a>

We pause to consider the Hsiao fit of the blue bands plotted above. We see that the chi-squared is dominated not by the morphology of the light-curve, but by the model's inability to match the supernova's color. This is problamatic since we would like to discriminate targets primarily based on their light-curve morphology. To address this we can fit each band independently without an overall color term.

This approach is what has been implimented in our analysis pipeline.

In [ ]:
prior = {'z': all_data.meta['redshift']}

fit_results = classification.run_classification_fits(
    demo_id, 
    all_data, 
    ['t0', 'amplitude', 'x1', 'c'], 
    simple_fit, 
    priors_hs=prior, 
    priors_bg=prior,
    kwargs_bg={'bounds': sn91bg_bounds},
    show_plots=True
)


In [ ]:
classification.classify_targets(fit_results, dr3.band_names, dr3.lambda_effective)
